In [111]:
import sys
import os
from datetime import datetime
from pathlib import Path

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter, defaultdict
import logging
from datetime import datetime
import traceback

from sklearn.model_selection import GridSearchCV, RepeatedStratifiedKFold, StratifiedKFold, LeaveOneOut
from sklearn.utils.multiclass import unique_labels

from sklearn.inspection import permutation_importance

from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import (
    RocCurveDisplay, auc, accuracy_score, precision_score, 
    recall_score, f1_score, confusion_matrix, classification_report,
    balanced_accuracy_score, roc_auc_score
)
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE
import xgboost as xgb
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.feature_selection import SelectKBest, f_classif
import re
import scipy



pd.set_option('display.max_columns', None)

sys.path.append(os.path.abspath("../src"))  

def bold_max(df, dataset="", precision=2):
    """
    Return a Styler that bolds the column-wise maxima.

    Works with both:
    - numeric values
    - strings in the format '0.84 ± 0.02'

    Parameters
    ----------
    df : pd.DataFrame
        DataFrame with either numeric or 'mean ± std' strings.
    dataset : str
        A caption or title to display above the table.
    precision : int, default 2
        Number of decimals to show if numeric.
    """
    def is_string_with_std(val):
        return isinstance(val, str) and '±' in val
    
    def is_string_with_bracket(val):
        return isinstance(val, str) and '[' in val

    if df.applymap(is_string_with_std).all().all():
        # All cells are strings with ±
        def highlight_max(col):
            means = col.str.extract(r"(\d+\.\d+) ±")[0].astype(float)
            max_val = means.max()
            return ['font-weight: bold' if v == max_val else '' for v in means]

        return df.style.set_caption(f"Dataset: {dataset}").apply(highlight_max, axis=0)

    else:
        # Assume numeric DataFrame
        return (
            df.style
              .set_caption(f"Dataset: {dataset}")
              .format(f"{{:.{precision}f}}")
              .apply(lambda col: ['font-weight: bold' if v == col.max() else '' for v in col], axis=0)
        )


In [2]:
processed_path = '../data/processed/'
# Read data
df_digital_tmt_with_target = pd.read_csv(processed_path + 'df_digital_tmt_with_target.csv') 
demographic_df = pd.read_csv(processed_path + 'demographic_df.csv') 
non_digital_df = pd.read_csv(processed_path + 'non_digital_df.csv') 
df_digital_hand_and_eye = pd.read_csv(processed_path + 'df_digital_hand_and_eye.csv') 
digital_test_less_subjects = pd.read_csv(processed_path + 'digital_test_less_subjects.csv') 
non_digital_test_less_subjects = pd.read_csv(processed_path + 'non_digital_test_less_subjects.csv') 


# Final checks
print(df_digital_tmt_with_target['group'].value_counts())
print(demographic_df['group'].value_counts())
print(non_digital_df['group'].value_counts())
print(df_digital_hand_and_eye['group'].value_counts())
print(digital_test_less_subjects['group'].value_counts())
print(non_digital_test_less_subjects['group'].value_counts())

group
1    42
0    37
Name: count, dtype: int64
group
1    42
0    37
Name: count, dtype: int64
group
1    42
0    37
Name: count, dtype: int64
group
1    30
0    26
Name: count, dtype: int64
group
1    30
0    26
Name: count, dtype: int64
group
1    30
0    26
Name: count, dtype: int64


## Training

In [110]:
# Configure logging
# Ensure logs folder exists
log_dir = "logs"
os.makedirs(log_dir, exist_ok=True)
# Create a fresh log file each run
log_filename = os.path.join(log_dir, f"error_log_{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.log")
logging.basicConfig(filename=log_filename,
                    level=logging.ERROR,
                    format='%(asctime)s - %(levelname)s - %(message)s')


# ───────────────────────────────────────────────────────────────
# 0. SET-UP GENERAL 
# ───────────────────────────────────────────────────────────────
n_splits = 2
n_repeats = 1

global_seed = 42
inner_cv_seed = 50  # Fixed for reproducibility in inner CV
perform_pca = False
type_of_csv = 'loo'
n_components = 4

datasets = ['demographic', 'demographic+digital','digital_test', 
            'non_digital_tests', 'non_digital_test_less_subjects', 
            'digital_test_less_subjects', 'hand_and_eye']

datasets = ['non_digital_test_less_subjects']
for value in [True, False]:
    perform_pca = value
    for dataset in datasets:
        try:
            print(f"Starting {dataset}: \n\n")
            if perform_pca:
                print("Performing PCA")

            match dataset:
                case 'demographic':
                    X = demographic_df.iloc[:, :-1].values
                    y = demographic_df.iloc[:, -1].values
                    feature_names = demographic_df.columns[:-1]
                case 'demographic+digital':
                    df_digital_plus_demo = df_digital_tmt_with_target.join(demographic_df.drop('group',axis=1))
                    df_digital_plus_demo = df_digital_plus_demo[[col for col in df_digital_plus_demo.columns if col != 'group'] + ['group']]
                    X = df_digital_plus_demo.iloc[:, :-1].values
                    y = df_digital_plus_demo.iloc[:, -1].values
                    feature_names = df_digital_plus_demo.columns[:-1]
                case 'non_digital_tests':
                    X = non_digital_df.iloc[:, :-1].values
                    y = non_digital_df.iloc[:, -1].values
                    feature_names = non_digital_df.columns[:-1]
                case 'non_digital_test_less_subjects':
                    X = non_digital_test_less_subjects.iloc[:, :-1].values
                    y = non_digital_test_less_subjects.iloc[:, -1].values
                    feature_names = non_digital_test_less_subjects.columns[:-1]
                case 'digital_test':
                    X = df_digital_tmt_with_target.iloc[:, :-1].values
                    y = df_digital_tmt_with_target.iloc[:, -1].values
                    feature_names = df_digital_tmt_with_target.columns[:-1]
                case 'digital_test_less_subjects':
                    X = digital_test_less_subjects.iloc[:, :-1].values
                    y = digital_test_less_subjects.iloc[:, -1].values
                    feature_names = digital_test_less_subjects.columns[:-1]
                case 'hand_and_eye':
                    X = df_digital_hand_and_eye.iloc[:, :-1].values
                    y = df_digital_hand_and_eye.iloc[:, -1].values
                    feature_names = df_digital_hand_and_eye.columns[:-1]
                case 'hand_and_eye_demo': # hand + eye + demo
                    df_hand_eye_plus_demo = df_digital_hand_and_eye.join(demographic_df.drop('group',axis=1))
                    df_hand_eye_plus_demo = df_hand_eye_plus_demo[[col for col in df_hand_eye_plus_demo.columns if col != 'group'] + ['group']]
                    X = df_hand_eye_plus_demo.iloc[:, :-1].values
                    y = df_hand_eye_plus_demo.iloc[:, -1].values
                    feature_names = df_hand_eye_plus_demo.columns[:-1]
                case _:
                    raise ValueError(f'please select a valid dataset from: {datasets}')

            # ───────────────────────────────────────────────────────────────
            # 1. DEFINICIÓN DE PARÁMETROS Y MODELOS 
            # ───────────────────────────────────────────────────────────────

            unique, counts = np.unique(y, return_counts=True)
            print("Class distribution:", dict(zip(unique, counts)))

            # Define parameter grids
            param_grids = {
                "RandomForestClassifier": {
                    "classifier__n_estimators": [100, 500, 700, 1000],
                    "classifier__max_depth": [None, 10, 20, 30]
                },
                "SVC": {
                    "classifier__C": [0.1, 1, 10],
                    "classifier__kernel": ['linear', 'rbf']
                },
                "LogisticRegression": {
                    "classifier__C": [0.1, 1, 10],
                    "classifier__penalty": ['l2']
                },
                "XGBClassifier": {
                    "classifier__n_estimators": [100, 300],
                    "classifier__max_depth": [3, 5],
                    "classifier__learning_rate": [0.05, 0.1]
                }
            }

            # Define models to evaluate
            models = [
                RandomForestClassifier(random_state=42, n_jobs=-1),
                SVC(random_state=42, probability=True),
                LogisticRegression(max_iter=1000, random_state=42, solver='saga', n_jobs=-1),
                xgb.XGBClassifier(random_state=42, tree_method="hist", eval_metric='logloss',n_jobs=-1)
            ]

            # ───────────────────────────────────────────────────────────────
            # 2. Cross validation
            # ───────────────────────────────────────────────────────────────

            match type_of_csv:
                case 'stratified':
                    print(f"RepeatedStratifiedKFold selected with n_splits = {n_splits} and n_repeats = {n_repeats}")
                    outer_cv = RepeatedStratifiedKFold(
                        n_splits=n_splits,
                        n_repeats=n_repeats,         
                        random_state=global_seed # Global seed
                    )
                case 'loo':
                    print("LeaveOneOut selected")
                    outer_cv = LeaveOneOut()
                case _:
                    print("select a valid CV type")

            mean_fpr = np.linspace(0, 1, 100)
            all_metrics_df = pd.DataFrame(columns=[
                'model', 'repeat', 'fold',   
                'accuracy', 'balanced_accuracy', 'precision', 
                'recall', 'f1', 'auc', 'specificity'
            ])

            # ───────────────────────────────────────────────────────────────
            # 3. External loop 
            # ───────────────────────────────────────────────────────────────
            for model in models:
                model_name = model.__class__.__name__
                print(f"\n🧪 CV for: {model_name}")

                tprs, aucs, best_params_list, fold_metrics = [], [], [], []
                feature_importance_counts = {n: 0 for n in feature_names}

                # fig, ax = plt.subplots(figsize=(6, 6))
                all_y_true, all_y_pred = [], []

                # Enumeramos 'repeat' y 'fold' para guardar en métricas
                for outer_idx, (train_idx, test_idx) in enumerate(outer_cv.split(X, y)):
                    fold = outer_idx  # index of the left-out observation
                    print('fold:', fold)

                    # ── Split
                    X_train, X_test = X[train_idx], X[test_idx]
                    y_train, y_test = y[train_idx], y[test_idx]

                    
                    # ── Inner CV: estratificado 3-fold con la MISMA semilla por repetición
                    inner_cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=inner_cv_seed)
                    
                    if perform_pca:
                        n_components = min(n_components, X_train.shape[1])
                        print("n_components:", n_components)
                        pca_step = ('pca', PCA(n_components=n_components))
                    else:
                        pca_step = ('noop', 'passthrough')

                    pipeline = Pipeline([
                        ('imputer', SimpleImputer(strategy='mean')),  # or 'median' depending on your data
                        ('scaler', StandardScaler()),
                        pca_step,
                        ('classifier', model)
                    ])

                    # Hiperparámetros
                    param_grid = param_grids.get(model_name, {})

                    grid = GridSearchCV(
                        pipeline,
                        param_grid=param_grid,
                        cv=inner_cv,               
                        scoring='roc_auc',
                        n_jobs=-1,
                        verbose=0
                    )
                    
                    grid.fit(X_train, y_train)
                    best_model = grid.best_estimator_
                    best_params_list.append(grid.best_params_)

                    # ── Predicción
                    y_pred_proba = best_model.predict_proba(X_test)[:, 1]
                    y_pred = best_model.predict(X_test)

                    all_y_true.extend(y_test)
                    all_y_pred.extend(y_pred)

                    fold_metrics.append({
                        'model': model_name,
                        'fold': fold,              
                        'y_test': y_test[0],
                        'y_pred': y_pred[0],
                        'y_pred_proba': y_pred_proba[0],
                        'feature_names': feature_names.values
                    })

                # ── Guardamos métricas
                all_metrics_df = pd.concat([all_metrics_df,
                                            pd.DataFrame(fold_metrics)],
                                        ignore_index=True)


            # save
            dir = f'./results/modelling/{datetime.now().strftime("%Y-%m-%d")}'
            os.makedirs(dir, exist_ok=True)
            if perform_pca:
                all_metrics_df.to_csv(f'{dir}/all_metrics_{dataset}_LOOCV_PCA_n_components{n_components}_{datetime.now().strftime("%s")[-4:]}.csv',index=False)
            else:
                all_metrics_df.to_csv(f'{dir}/all_metrics_{dataset}_LOOCV_{datetime.now().strftime("%s")[-4:]}.csv',index=False)
        except Exception as e:
            error_msg = traceback.format_exc().strip().split("\n")[-1]  # only last line of error
            logging.error(f"[{dataset}] PCA={perform_pca} → {error_msg}")
            print(f"⚠️ An error occurred with dataset {dataset}. Check log file: {log_filename}")


Starting non_digital_test_less_subjects: 


Performing PCA
Class distribution: {0: 26, 1: 30}
LeaveOneOut selected

🧪 CV for: RandomForestClassifier
fold: 0
n_components: 4


fold: 1
n_components: 4
fold: 2
n_components: 4
fold: 3
n_components: 4
fold: 4
n_components: 4
fold: 5
n_components: 4
fold: 6
n_components: 4
fold: 7
n_components: 4
fold: 8
n_components: 4
fold: 9
n_components: 4
fold: 10
n_components: 4
fold: 11
n_components: 4
fold: 12
n_components: 4
fold: 13
n_components: 4
fold: 14
n_components: 4
fold: 15
n_components: 4
fold: 16
n_components: 4
fold: 17
n_components: 4
fold: 18
n_components: 4
fold: 19
n_components: 4
fold: 20
n_components: 4
fold: 21
n_components: 4
fold: 22
n_components: 4
fold: 23
n_components: 4
fold: 24
n_components: 4
fold: 25
n_components: 4
fold: 26
n_components: 4
fold: 27
n_components: 4
fold: 28
n_components: 4
fold: 29
n_components: 4
fold: 30
n_components: 4
fold: 31
n_components: 4
fold: 32
n_components: 4
fold: 33
n_components: 4
fold: 34
n_components: 4
fold: 35
n_components: 4
fold: 36
n_components: 4
fold: 37
n_components: 4
fold: 38
n_components: 4
fold: 39
n_components: 4
fold: 40
n_components: 4
fold: 41


In [105]:
from sklearn.decomposition import PCA

# Fit PCA on full data
pca_full = PCA(n_components=4).fit(X)

# Fit PCA on training sets in each LOOCV split and compare
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics.pairwise import cosine_similarity

X = df_digital_tmt_with_target.iloc[:, :-1].values
from sklearn.feature_selection import VarianceThreshold

selector = VarianceThreshold(threshold=0.01)  # Tune threshold if needed
X_reduced = selector.fit_transform(X)

def remove_highly_correlated(df, threshold=0.95):
    corr_matrix = pd.DataFrame(df).corr().abs()
    upper = corr_matrix.where(
        np.triu(np.ones(corr_matrix.shape), k=1).astype(bool)
    )
    to_drop = [column for column in upper.columns if any(upper[column] > threshold)]
    return pd.DataFrame(df).drop(columns=to_drop)

X_reduced = remove_highly_correlated(X)


loo = LeaveOneOut()
cos_sims = []

for train_idx, test_idx in loo.split(X_reduced):
    X_train = X[train_idx]
    pca_cv = PCA(n_components=4).fit(X_train)
    cos = cosine_similarity(pca_cv.components_, pca_full.components_).mean()
    cos_sims.append(cos)

print(f"Average cosine similarity between LOOCV PCA and full PCA: {np.mean(cos_sims):.4f}")


Average cosine similarity between LOOCV PCA and full PCA: 0.2499


In [102]:
n_components = 4
X = df_digital_tmt_with_target.iloc[:, :-1].values
min(n_components, X.shape[1])
n_components

4

In [24]:
processed_path
all_metrics_df2 = pd.read_csv('./results/modelling/2025-06-11/all_metrics_demographic_LOOCV_1749.csv')
all_metrics_df2['y_pred'].sum()

191.0

In [ ]:

def calculate_metrics_leave_one_out(df, model_name):
    y_test_all = [i[0] for i in df[df['model'] == model_name]['y_test'].values]
    y_pred_proba_all = [i[0] for i in df[df['model'] == model_name]['y_pred_proba'].values]
    roc_auc_score(y_test_all, y_pred_proba_all)
    return pd.DataFrame({
            'model': model_name,  
            'auc': roc_auc_score(y_test_all, y_pred_proba_all),        
            # 'accuracy': accuracy_score(y_test_all, y_pred_proba_all),
            # 'balanced_accuracy': balanced_accuracy_score(y_test_all, y_pred_proba_all),
            # 'precision': precision_score(y_test_all, y_pred_proba_all),
            # 'recall': recall_score(y_test_all, y_pred_proba_all),
            # 'f1': f1_score(y_test_all, y_pred_proba_all),
            }, index=[0])

model_dfs = []
for model_name in all_metrics_df['model'].unique():
    model_dfs.append(calculate_metrics_leave_one_out(all_metrics_df, model_name))

pd.concat(model_dfs).reset_index(drop=True)

['digital_test_less_subjects', 'digital_test', 'demographic+digital', 'hand_and_eye', 'digital_test', 'demographic', 'demographic+digital', 'demographic']


In [112]:
def calculate_metrics_leave_one_out(df, model_name):
    y_true = df[df['model'] == model_name]['y_test'].tolist()
    y_pred_proba = df[df['model'] == model_name]['y_pred_proba'].tolist()
    # y_pred = [1 if p >= 0.5 else 0 for p in y_pred_proba]  # Binarize
    y_pred = df[df['model'] == model_name]['y_pred'].tolist()

    try:
        auc = roc_auc_score(y_true, y_pred_proba)
    except ValueError:
        auc = np.nan

    return pd.DataFrame({
        'model': [model_name],
        'auc': [auc],
        'accuracy': [accuracy_score(y_true, y_pred)],
        'balanced_accuracy': [balanced_accuracy_score(y_true, y_pred)],
        'precision': [precision_score(y_true, y_pred, zero_division=0)],
        'recall': [recall_score(y_true, y_pred, zero_division=0)],
        'f1': [f1_score(y_true, y_pred, zero_division=0)]
    })


data_files = os.listdir('./results/modelling/2025-06-11') + os.listdir('./results/modelling/2025-06-12')

# Define the two directories
dir1 = Path('./results/modelling/2025-06-11')
dir2 = Path('./results/modelling/2025-06-12')

# Get all files recursively in both directories
files = list(dir1.rglob('*')) + list(dir2.rglob('*'))

# Filter only files (exclude directories) and convert to absolute paths
file_paths = [f.resolve() for f in files if f.is_file()]

all_dataset_metrics = []
# Print or use the paths
for path in file_paths:
    print(path)
    pattern = re.compile(r"all_metrics_(.*?)_LOOCV")
    match = pattern.search(path.stem)
    dataset = match.group(1)
    print(dataset)

    all_metrics_df = pd.read_csv(path)
    model_dfs = [calculate_metrics_leave_one_out(all_metrics_df, model_name)
                for model_name in all_metrics_df['model'].unique()]

    metrics_global = pd.concat(model_dfs, ignore_index=True)
    metrics_global['dataset'] = dataset
    metrics_global['PCA'] = True if 'PCA' in str(path) else False
    

    all_dataset_metrics.append(metrics_global)

/home/gus/Documents/REPOS/tmt-analysis/notebooks/results/modelling/2025-06-11/all_metrics_digital_test_less_subjects_LOOCV_PCA_n_components3_1749.csv
digital_test_less_subjects
/home/gus/Documents/REPOS/tmt-analysis/notebooks/results/modelling/2025-06-11/all_metrics_digital_test_LOOCV_1749.csv
digital_test
/home/gus/Documents/REPOS/tmt-analysis/notebooks/results/modelling/2025-06-11/all_metrics_demographic+digital_LOOCV_PCA_n_components3_1749.csv
demographic+digital
/home/gus/Documents/REPOS/tmt-analysis/notebooks/results/modelling/2025-06-11/all_metrics_hand_and_eye_LOOCV_PCA_n_components3_1749.csv
hand_and_eye
/home/gus/Documents/REPOS/tmt-analysis/notebooks/results/modelling/2025-06-11/all_metrics_digital_test_LOOCV_PCA_n_components3_1749.csv
digital_test
/home/gus/Documents/REPOS/tmt-analysis/notebooks/results/modelling/2025-06-11/all_metrics_demographic_LOOCV_1749.csv
demographic
/home/gus/Documents/REPOS/tmt-analysis/notebooks/results/modelling/2025-06-11/all_metrics_demographic+

In [113]:
all_datasets_df = pd.concat(all_dataset_metrics)
all_datasets_df.groupby(['model', 'dataset', 'PCA'])
top1_per_dataset = all_datasets_df.groupby(['dataset', 'PCA'], group_keys=False).apply(
    lambda group: group.nlargest(1, 'auc')
)
top1_per_dataset.sort_values('auc', ascending=False)

,model,auc,accuracy,balanced_accuracy,precision,recall,f1,dataset,PCA
1,SVC,0.857692,0.660714,0.650000,0.648649,0.800000,0.716418,non_digital_test_less_subjects,True
2,LogisticRegression,0.814103,0.767857,0.767949,0.793103,0.766667,0.779661,non_digital_test_less_subjects,False
2,LogisticRegression,0.764479,0.721519,0.723616,0.763158,0.690476,0.725000,non_digital_tests,True
2,LogisticRegression,0.756757,0.721519,0.723616,0.763158,0.690476,0.725000,non_digital_tests,False
2,LogisticRegression,0.667310,0.632911,0.630631,0.651163,0.666667,0.658824,demographic+digital,False
2,LogisticRegression,0.648649,0.607595,0.605212,0.627907,0.642857,0.635294,digital_test,False
3,XGBClassifier,0.612613,0.632911,0.627413,0.638298,0.714286,0.674157,demographic,False
2,LogisticRegression,0.601282,0.553571,0.555128,0.592593,0.533333,0.561404,digital_test_less_subjects,False
2,LogisticRegression,0.560489,0.531646,0.527349,0.555556,0.595238,0.574713,demographic+digital,True
2,LogisticRegression,0.559202,0.531646,0.527349,0.555556,0.595238,0.574713,digital_test,True


In [95]:
all_datasets_df.sort_values('auc', ascending=False)

,model,auc,accuracy,balanced_accuracy,precision,recall,f1,dataset,PCA
2,LogisticRegression,0.667310,0.632911,0.630631,0.651163,0.666667,0.658824,demographic+digital,False
2,LogisticRegression,0.648649,0.607595,0.605212,0.627907,0.642857,0.635294,digital_test,False
1,SVC,0.640927,0.594937,0.591699,0.613636,0.642857,0.627907,digital_test,False
3,XGBClassifier,0.612613,0.632911,0.627413,0.638298,0.714286,0.674157,demographic,False
2,LogisticRegression,0.601282,0.553571,0.555128,0.592593,0.533333,0.561404,digital_test_less_subjects,False
1,SVC,0.595238,0.594937,0.593308,0.619048,0.619048,0.619048,demographic+digital,False
3,XGBClassifier,0.583012,0.569620,0.566281,0.590909,0.619048,0.604651,digital_test,False
3,XGBClassifier,0.579794,0.607595,0.605212,0.627907,0.642857,0.635294,demographic+digital,False
0,RandomForestClassifier,0.572716,0.493671,0.491634,0.523810,0.523810,0.523810,demographic,False
0,RandomForestClassifier,0.569498,0.506329,0.503539,0.534884,0.547619,0.541176,digital_test,False


In [118]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re

def plot_model_metrics(path):
    all_metrics_df = pd.read_csv(path)

    # Extraer nombre del dataset
    match = re.search(r"(?<=all_metrics_).*(?=_nested4x10\.csv)", path)
    dataset = match.group() if match else "unknown"
    print("Dataset name:", dataset)

    # Tabla de medias por modelo
    metrics_comparison = all_metrics_df.groupby('model').mean(numeric_only=True)
    # display(bold_max(metrics_comparison, dataset=dataset))

    # Crear tabla con formato "mean [min max]" por modelo y métrica
    metrics_to_plot = ['balanced_accuracy', 'precision', 'recall', 'f1', 'auc']
    summary_formatted = []

    for model in all_metrics_df['model'].unique():
        row = {'model': model}
        df_model = all_metrics_df[all_metrics_df['model'] == model]
        for metric in metrics_to_plot:
            mean_val = df_model[metric].mean()
            min_val = df_model[metric].min()
            max_val = df_model[metric].max()
            row[metric] = f"{mean_val:.3f} [{min_val:.3f} {max_val:.3f}]"
        summary_formatted.append(row)

    formatted_df = pd.DataFrame(summary_formatted)
    print("\nResumen de métricas por modelo (mean [min max]):")
    display(formatted_df)

    # Boxplot
    df_long = all_metrics_df.melt(
        id_vars=['model', 'repeat', 'fold'],
        value_vars=metrics_to_plot,
        var_name='metric',
        value_name='score'
    )

    plt.figure(figsize=(14, 6))
    sns.boxplot(
        data=df_long,
        x='metric',
        y='score',
        hue='model',
        linewidth=1.5
    )

    formatted_df['dataset'] = dataset
    plt.axhline(0.5, color='gray', linestyle='--', linewidth=1.4)
    plt.title(f'Distribución de métricas para el dataset "{dataset}"', fontsize=16)
    plt.xlabel('Métrica', fontsize=14)
    plt.ylabel('Score', fontsize=14)
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)
    plt.legend(title='Modelo', fontsize=10, title_fontsize=8, loc='best', bbox_to_anchor=(0, 0, 1, 1))
    plt.tight_layout()
    plt.show()

    return formatted_df


In [ ]:
dfs_results = []
for dataset in ['pca', 'demographic', 'digital_test', 'demographic+digital', 'hand_and_eye', 'non_digital_tests', 'digital_test_less_subjects', 'hand_and_eye_demo']:
    df_res = plot_model_metrics(f'/home/gus/Documents/REPOS/tmt-analysis/notebooks/results/modelling/resultados_seminario_06_06_25/all_metrics_{dataset}_nested4x10.csv')
    dfs_results.append(df_res)